In [1]:
import numpy as np
import pandas as pd
from fmskill import ModelResult, PointObservation, TrackObservation, Connector
%load_ext autoreload
%autoreload 2
%matplotlib inline

## Observations

In [2]:
o1 = PointObservation('../tests/testdata/SW/HKNA_Hm0.dfs0', item=0, x=4.2420, y=52.6887, name="HKNA")
o2 = PointObservation("../tests/testdata/SW/eur_Hm0.dfs0", item=0, x=3.2760, y=51.9990, name="EPL")
o3 = TrackObservation("../tests/testdata/SW/Alti_c2_Dutch.dfs0", item=3, name="c2")

## MIKE ModelResult

In [3]:
mr1 = ModelResult('../tests/testdata/SW/HKZN_local_2017_DutchCoast.dfsu', name='SW_1', item=0)

## NetCDF ModelResult

In [4]:
fn = r"..\tests\testdata\SW\ERA5_DutchCoast.nc"

In [5]:
mr = ModelResult(fn)

In [6]:
mr

<XArrayModelResult> 'ERA5_DutchCoast'
- Item: 0: mwd
- Item: 1: mwp
- Item: 2: mp2
- Item: 3: pp1d
- Item: 4: swh

In [7]:
mr.ds  # mr contains the xr.Dataset

<xarray.Dataset>
Dimensions:  (time: 67, x: 20, y: 11)
Coordinates:
  * x        (x) float32 -1.0 -0.5 0.0 0.5 1.0 1.5 ... 6.0 6.5 7.0 7.5 8.0 8.5
  * y        (y) float32 55.0 54.5 54.0 53.5 53.0 52.5 52.0 51.5 51.0 50.5 50.0
  * time     (time) datetime64[ns] 2017-10-27 ... 2017-10-29T18:00:00
Data variables:
    mwd      (time, y, x) float32 ...
    mwp      (time, y, x) float32 ...
    mp2      (time, y, x) float32 ...
    pp1d     (time, y, x) float32 ...
    swh      (time, y, x) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2021-06-07 12:25:02 GMT by grib_to_netcdf-2.16.0: /opt/ecmw...

## ModelResultItem
We can select an item from the NetCDF file be using square brackets refering either to the item name or number. 

In [8]:
mr['swh']

<XArrayModelResultItem> 'ERA5_DutchCoast'
- Item: swh

## Test extract from XArray

In [16]:
mr['swh']._extract_point(o1).head()

,x,y,swh
time,,,
2017-10-27 00:00:00,4.242,52.6887,1.220337
2017-10-27 01:00:00,4.242,52.6887,1.346573
2017-10-27 02:00:00,4.242,52.6887,1.465747
2017-10-27 03:00:00,4.242,52.6887,1.611757
2017-10-27 04:00:00,4.242,52.6887,1.792901


In [13]:
mr['swh'].extract_observation(o1)  # will call _extract_point

<PointComparer>
Observation: HKNA, n_points=386
 Model: swh, rmse=0.545

In [28]:
mr['swh']._extract_track(o3).head()

,x,y,swh
time,,,
2017-10-27 12:52:52.337,2.422854,51.253353,1.438809
2017-10-27 12:52:53.280,2.413789,51.310268,1.464191
2017-10-27 12:52:54.224,2.404711,51.367184,1.489200
2017-10-27 12:52:55.167,2.395619,51.424099,1.513833
2017-10-27 12:52:56.111,2.386516,51.481014,1.538092


## Connect

In [25]:
con = Connector([o1,o2,o3], mr['swh'])

In [26]:
cc = con.extract()

In [27]:
cc.skill()

,n,bias,rmse,urmse,mae,cc,si,r2
observation,,,,,,,,
EPL,67,-0.205197,0.287825,0.201836,0.220975,0.972297,0.074036,0.888310
HKNA,386,-0.437425,0.545329,0.325643,0.441374,0.974863,0.098380,0.772663
c2,298,-0.567862,0.694198,0.399303,0.597021,0.936752,0.115172,0.629237
